In [50]:
#adaptado de https://github.com/GrupoTuringCodes/fernando-pessoa/blob/master/analise.ipynb

import pandas as pd
import nltk
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import model_selection, naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem.snowball import PortugueseStemmer
import spacy
import re
import string
from unidecode import unidecode
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [29]:
data = pd.read_csv("final.csv")
data = data.drop('Unnamed: 0', axis=1)
data.head()

,review_rate,review_body
0,10,"Pois bem...as fotos dos pratos, bebidas e doce..."
1,30,Espero que utilizem essa avaliação para rever ...
2,20,"Fomos comer a sobremesa as 20h40, sentamos, pe..."
3,10,Pedimos o cardápio e ao chamar o atendente ped...
4,50,"Fui com algumas amigas em uma segunda-feira, d..."


In [30]:
def DATA_BINARIZADOR_RESULTADOS(data):
    RESULTADO_BINARIO = []
    
    for item in data['review_rate']:
        if item < 40 :
            RESULTADO_BINARIO.append(0)
        elif item >= 40:
            RESULTADO_BINARIO.append(1)
        else:
            print("algo deu ruim")
            
    
    data['RESULTADO_BINARIO'] = RESULTADO_BINARIO
    return data


In [31]:
data = DATA_BINARIZADOR_RESULTADOS(data)
data.head(50)

,review_rate,review_body,RESULTADO_BINARIO
0,10,"Pois bem...as fotos dos pratos, bebidas e doce...",0
1,30,Espero que utilizem essa avaliação para rever ...,0
2,20,"Fomos comer a sobremesa as 20h40, sentamos, pe...",0
3,10,Pedimos o cardápio e ao chamar o atendente ped...,0
4,50,"Fui com algumas amigas em uma segunda-feira, d...",1
5,50,"Quem gosta de doce, é um prato cheio. Erramos ...",1
6,30,"Atendimento muito bom, confesso que quase peca...",0
7,10,"As sobremesas são realmente uma delícia, porém...",0
8,50,O bar oferece diversas opções de comidinhas de...,1
9,50,"Ao chegar, o local já agrada. Tanto o visual, ...",1


In [32]:
def clear_text(text):
    #remove pontuacao, palavras com numeros, deixa o texto em caixa baixa e remove o texto entre colchetes
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = unidecode(text)
    return text

clear = lambda x: clear_text(x)

In [33]:
data_clean = pd.DataFrame(data.review_body.apply(clear))
data_clean

,review_body
0,pois bemas fotos dos pratos bebidas e doces sa...
1,espero que utilizem essa avaliacao para rever ...
2,fomos comer a sobremesa as sentamos pedimos e...
3,pedimos o cardapio e ao chamar o atendente ped...
4,fui com algumas amigas em uma segundafeira dia...
...,...
7667,com amigos sozinho ou em familia achei o local...
7668,pratos arabes muito bons e variados otimo aten...
7669,bem ao estilo arabe o dono e muito querido e s...
7670,zaki e uma especie de restaurante e loja de pr...


In [34]:
def remover_underscore(palavra):
    if palavra[0] == '_' or palavra[-1] == '_':
        return palavra.replace('_', '')
    else:
        return palavra

In [35]:
def stemizar(texto_original):
    palavras_importantes = remover_stopwords(texto_original, lista=True)
    
    stemizador = PortugueseStemmer()
    palavras = []

    for p in palavras_importantes:
        novo_p = remover_underscore(remover_acento(p))
        palavras.append(stemizador.stem(novo_p))
    return ' '.join(palavras)

In [36]:
def remover_acento(texto):
    try:
        texto = unicode(texto, 'utf-8')
    except NameError:
        pass
    texto = unicodedata.normalize('NFD', texto)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(texto)

In [37]:
stopwords_pt = set(stopwords.words('portuguese'))

def remover_stopwords(texto, lista=False):
    palavras = re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())
    sem_stopwords = [p for p in palavras if p not in stopwords_pt]
    if lista:
        return sem_stopwords
    else:
        return ' '.join(sem_stopwords)

In [39]:
data_clean['texto_stemizado'] = data_clean.review_body.apply(stemizar)
data_clean['texto_l_s'] = data_clean.review_body.apply(lambda x: stemizar(x))
data_clean['texto_sem_stopwords'] = data_clean.review_body.apply(remover_stopwords)

In [41]:
data_clean.head()

,review_body,texto_stemizado,texto_l_s,texto_sem_stopwords
0,pois bemas fotos dos pratos bebidas e doces sa...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bemas fotos pratos bebidas doces sao mara...
1,espero que utilizem essa avaliacao para rever ...,esper utiliz avaliaca rev conceit negoci poten...,esper utiliz avaliaca rev conceit negoci poten...,espero utilizem avaliacao rever conceito negoc...
2,fomos comer a sobremesa as sentamos pedimos e...,com sobremes sent ped esper acredit detalh sob...,com sobremes sent ped esper acredit detalh sob...,comer sobremesa sentamos pedimos esperamos acr...
3,pedimos o cardapio e ao chamar o atendente ped...,ped cardapi cham atendent ped prat pass nao di...,ped cardapi cham atendent ped prat pass nao di...,pedimos cardapio chamar atendente pedimos prat...
4,fui com algumas amigas em uma segundafeira dia...,algum amig segundafeir dia promoca doc apes es...,algum amig segundafeir dia promoca doc apes es...,algumas amigas segundafeira dia promocao doces...


In [42]:
data_clean['escore'] = data['RESULTADO_BINARIO']

In [43]:
data_clean.head()

,review_body,texto_stemizado,texto_l_s,texto_sem_stopwords,escore
0,pois bemas fotos dos pratos bebidas e doces sa...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bemas fotos pratos bebidas doces sao mara...,0
1,espero que utilizem essa avaliacao para rever ...,esper utiliz avaliaca rev conceit negoci poten...,esper utiliz avaliaca rev conceit negoci poten...,espero utilizem avaliacao rever conceito negoc...,0
2,fomos comer a sobremesa as sentamos pedimos e...,com sobremes sent ped esper acredit detalh sob...,com sobremes sent ped esper acredit detalh sob...,comer sobremesa sentamos pedimos esperamos acr...,0
3,pedimos o cardapio e ao chamar o atendente ped...,ped cardapi cham atendent ped prat pass nao di...,ped cardapi cham atendent ped prat pass nao di...,pedimos cardapio chamar atendente pedimos prat...,0
4,fui com algumas amigas em uma segundafeira dia...,algum amig segundafeir dia promoca doc apes es...,algum amig segundafeir dia promoca doc apes es...,algumas amigas segundafeira dia promocao doces...,1


In [46]:
to_x, te_x, to_y, te_y = train_test_split(data_clean[['review_body', 'texto_stemizado', 'texto_l_s', 'texto_sem_stopwords']], data_clean['escore'], test_size=0.2, random_state=42)

In [47]:
tfidf_vector = TfidfVectorizer()
tfidf_vector.fit(data_clean['texto_l_s'])
train_X_tfidf = tfidf_vector.transform(to_x['texto_l_s'])
test_X_tfidf = tfidf_vector.transform(te_x['texto_l_s'])

data_clean_tfidf = tfidf_vector.transform(data_clean['texto_l_s'])

In [53]:
nb = GaussianNB()
nb.fit(train_X_tfidf.toarray(), to_y)
predictions_nb = nb.predict(test_X_tfidf.toarray())
acuracia = accuracy_score(predictions_nb, te_y)
print(f"Acurácia do Naive Bayes: {acuracia*100:.3f}%")

Acurácia do Naive Bayes: 56.678%


In [54]:
svm = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm.fit(train_X_tfidf, to_y)
predictions_svm = svm.predict(test_X_tfidf)
acuracia = accuracy_score(predictions_svm, te_y)
print(f"Acurácia da SVM: {acuracia*100:.3f}%")

Acurácia da SVM: 92.052%


In [55]:
#mudando pras notas:
data_clean['escore'] = data['review_rate']
to_x, te_x, to_y, te_y = train_test_split(data_clean[['review_body', 'texto_stemizado', 'texto_l_s', 'texto_sem_stopwords']], data_clean['escore'], test_size=0.2, random_state=42)

In [56]:
data_clean.head()

,review_body,texto_stemizado,texto_l_s,texto_sem_stopwords,escore
0,pois bemas fotos dos pratos bebidas e doces sa...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bem fot prat beb doc sao maravilhosasdaqu...,pois bemas fotos pratos bebidas doces sao mara...,10
1,espero que utilizem essa avaliacao para rever ...,esper utiliz avaliaca rev conceit negoci poten...,esper utiliz avaliaca rev conceit negoci poten...,espero utilizem avaliacao rever conceito negoc...,30
2,fomos comer a sobremesa as sentamos pedimos e...,com sobremes sent ped esper acredit detalh sob...,com sobremes sent ped esper acredit detalh sob...,comer sobremesa sentamos pedimos esperamos acr...,20
3,pedimos o cardapio e ao chamar o atendente ped...,ped cardapi cham atendent ped prat pass nao di...,ped cardapi cham atendent ped prat pass nao di...,pedimos cardapio chamar atendente pedimos prat...,10
4,fui com algumas amigas em uma segundafeira dia...,algum amig segundafeir dia promoca doc apes es...,algum amig segundafeir dia promoca doc apes es...,algumas amigas segundafeira dia promocao doces...,50


In [57]:
nb = GaussianNB()
nb.fit(train_X_tfidf.toarray(), to_y)
predictions_nb = nb.predict(test_X_tfidf.toarray())
acuracia = accuracy_score(predictions_nb, te_y)
print(f"Acurácia do Naive Bayes: {acuracia*100:.3f}%")

Acurácia do Naive Bayes: 32.508%


In [58]:
svm = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm.fit(train_X_tfidf, to_y)
predictions_svm = svm.predict(test_X_tfidf)
acuracia = accuracy_score(predictions_svm, te_y)
print(f"Acurácia da SVM: {acuracia*100:.3f}%")

Acurácia da SVM: 66.384%
